<a href="https://colab.research.google.com/github/gayathri8799/Sentiment-Analysis-Using-Custom-LSTM-and-Pretrained-AWD-LSTM-ULMFiT-/blob/main/sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers datasets scikit-learn matplotlib

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import re

from datasets import load_dataset
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
# LOAD IMDb DATASET
dataset = load_dataset("imdb")

train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]

test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#lstm
# Text Preprocessing
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.split()

In [ ]:
MAX_VOCAB = 20000
MAX_LEN = 300

PAD = "<pad>"
UNK = "<unk>"

counter = Counter()
for text in train_texts:
    counter.update(tokenize(text))

vocab = {PAD: 0, UNK: 1}
for word, _ in counter.most_common(MAX_VOCAB - 2):
    vocab[word] = len(vocab)

In [ ]:
# encoding
def encode(text):
    tokens = tokenize(text)
    ids = [vocab.get(t, vocab[UNK]) for t in tokens][:MAX_LEN]
    return ids + [0] * (MAX_LEN - len(ids))

In [ ]:
#dataset
class LSTMDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            torch.tensor(encode(self.texts[idx]), dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

In [ ]:
# lstm model
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 128, padding_idx=0)
        self.lstm = nn.LSTM(128, 256, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(256, 2)

    def forward(self, x):
        x = self.embedding(x)
        _, (h, _) = self.lstm(x)
        return self.fc(self.dropout(h[-1]))

In [ ]:
# train and evaluate
train_loader = DataLoader(
    LSTMDataset(train_texts, train_labels),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    LSTMDataset(test_texts, test_labels),
    batch_size=64
)

lstm_model = SentimentLSTM(len(vocab)).to(device)
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_lstm():
    lstm_model.train()
    for epoch in range(5):
        loss_sum = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(lstm_model(x), y)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
        print(f"LSTM Epoch {epoch+1} Loss: {loss_sum/len(train_loader):.4f}")

In [ ]:
def eval_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            preds.extend(model(x).argmax(1).cpu().numpy())
            labels.extend(y.numpy())
    acc = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return acc, p, r, f

In [ ]:
train_lstm()
lstm_results = eval_model(lstm_model, test_loader)

LSTM Epoch 1 Loss: 0.6931
LSTM Epoch 2 Loss: 0.6870
LSTM Epoch 3 Loss: 0.6694
LSTM Epoch 4 Loss: 0.6527
LSTM Epoch 5 Loss: 0.6221


In [ ]:
# AWD-LSTM (ULMFiT)
from fastai.text.all import *

# Load IMDb dataset
path = untar_data(URLs.IMDB)

dls = TextDataLoaders.from_folder(
    path, train='train', valid='test', bs=64
)

# Create learner
awd_learner = text_classifier_learner(
    dls, AWD_LSTM, metrics=accuracy
)

# Fine-tune
awd_learner.fit_one_cycle(1, 2e-2)
awd_learner.unfreeze()
awd_learner.fit_one_cycle(2, slice(1e-4, 1e-3))

# Get accuracy (float)
awd_acc = awd_learner.validate()[1]
print("AWD-LSTM Accuracy:", awd_acc)

epoch,train_loss,valid_loss,accuracy,time
0,0.432329,0.372677,0.835760,03:23


epoch,train_loss,valid_loss,accuracy,time
0,0.291515,0.237536,0.901880,07:26
1,0.248820,0.210534,0.918040,07:27


AWD-LSTM Accuracy: 0.9180399775505066


In [ ]:
preds, labels = awd_learner.get_preds()
pred_labels = preds.argmax(dim=1)
awd_acc = accuracy_score(labels, pred_labels)
awd_prec, awd_rec, awd_f1, _ = precision_recall_fscore_support(labels, pred_labels, average='binary')
print(f"AWD-LSTM Results: Accuracy={awd_acc:.4f}, Precision={awd_prec:.4f}, Recall={awd_rec:.4f}, F1={awd_f1:.4f}")

AWD-LSTM Results: Accuracy=0.9180, Precision=0.9206, Recall=0.9150, F1=0.9178


In [ ]:
# BERT TRANSFORMER
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def bert_encode(texts):
    return tokenizer(
        texts, padding=True, truncation=True,
        max_length=256, return_tensors="pt"
    )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# dataset
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
from transformers import BertTokenizer
from torch.optim import AdamW

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = BERTDataset(train_texts, train_labels, tokenizer)
test_dataset = BERTDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
def train_bert(model, loader, optimizer):
    model.train()
    total_loss = 0

    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
    loss = train_bert(model, train_loader, optimizer)
    print(f"BERT Epoch {epoch+1}/{EPOCHS} | Loss: {loss:.4f}")

BERT Epoch 1/3 | Loss: 0.2679
BERT Epoch 2/3 | Loss: 0.1411
BERT Epoch 3/3 | Loss: 0.0719


In [ ]:
def evaluate_bert(model, loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in loader:
            labels.extend(batch["labels"].numpy())
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds.extend(outputs.logits.argmax(1).cpu().numpy())

    acc = accuracy_score(labels, preds)
    p, r, f, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return acc, p, r, f

In [ ]:
bert_results = evaluate_bert(model, test_loader)
print("BERT Results:", bert_results)

BERT Results: (0.9182, 0.9229026777768788, 0.91264, 0.9177426491291582)


In [ ]:
print("\nFINAL COMPARISON\n")

print("Custom LSTM  :", lstm_results)
print("AWD-LSTM Acc :", awd_acc)
print("BERT         :", bert_results)


FINAL COMPARISON

Custom LSTM  : (0.5098, 0.5542755870624723, 0.10008, 0.16954665582435455)
AWD-LSTM Acc : 0.9180399775505066
BERT         : (0.9182, 0.9229026777768788, 0.91264, 0.9177426491291582)


In [ ]:
print("\nFINAL COMPARISON TABLE\n")
print(f"{'Model':<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1':<10}")
print(f"{'Custom LSTM':<15} {lstm_results[0]:<10.4f} {lstm_results[1]:<10.4f} {lstm_results[2]:<10.4f} {lstm_results[3]:<10.4f}")
print(f"{'AWD-LSTM':<15} {awd_acc:<10.4f} {awd_prec:<10.4f} {awd_rec:<10.4f} {awd_f1:<10.4f}")
print(f"{'BERT':<15} {bert_results[0]:<10.4f} {bert_results[1]:<10.4f} {bert_results[2]:<10.4f} {bert_results[3]:<10.4f}")


FINAL COMPARISON TABLE

Model           Accuracy   Precision  Recall     F1        
Custom LSTM     0.5098     0.5543     0.1001     0.1695    
AWD-LSTM        0.9180     0.9206     0.9150     0.9178    
BERT            0.9182     0.9229     0.9126     0.9177    
